In [35]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf
from datetime import datetime

In [36]:
def fetch_stocks():
    # filters = ['ind_exchangetradedfund','sh_avgvol_o2000']
    filters = ['sh_avgvol_o2000', 'sh_price_o20']
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [37]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:12<00:00,  2.54it/s]


In [38]:
def cal_VRRS(ticker, ref, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * 100

In [39]:
ticker_dict = {}

In [ ]:
start_date = "2017-01-01"
# end_date = "2022-05-30"

end_date = datetime.today().date()

spy = yf.Ticker("SPY")
spy = spy.history(start=start_date, end=end_date, interval='5d')['Close'].dropna()

for stock in stock_list:
    
    ticker = stock['Ticker']

    stock = yf.Ticker(ticker)    
    stock = stock.history(start=start_date, end=end_date, interval='5d')['Close'].dropna()

    # Calculate the VRRS indicator:
    rolling_length = 21 
    
    vrrs = cal_VRRS(stock, spy, rolling_length)
    
    ticker_dict[ticker] = vrrs.iloc[-1]
    
    print(ticker, vrrs.iloc[-1])

VET 7.891684137999364
MGY -9.047672132436686
PINS 0.29501504612621243
ASAN -26.77989489939549
IRT -7.6573647801259455
BRX -6.525055875934531
DBA 1.413724494078736
TBF 15.813331373271545
ELAN -8.654817243008335
BKLN 3.865303356954068
UNG 3.2123462242906036
CVET 22.91231205404537
T nan
BTU -3.3420959117311204
GPK 10.441589674412254
PATH 0.41479554793636825
JWN -5.058506924363821
DUST 43.45022507174698
ELY 2.2368580084466134
EWA -1.947658009055976
AES 5.12911499268213
STWD 2.367863482656016
PHG -13.852742392915635
NOVA 15.892059933380553
MRO -4.287281968859608
LAC -11.241811882766985
SIX -32.23927235876978
FHN 8.618931767598086
VXX -4.021752260312507
EXEL 14.128951162110118
EWH 6.918256422200553
VST 1.7924484015990962
MT -16.9647346854555
LBTYK -4.596754448435006
LTHM -2.045303435103084
MNDT 12.314014644854515
CCJ -2.5522382652283655
UBER -15.2129436347814
SBLK -8.108681444001672
TPX -15.907676729101672
ILF -4.772765992656423
NIO 23.616871088133113
EWG -7.80999516961612
DBX 10.64090343819

PDD 34.82938597103511
VGIT 4.652209179720929
XRT -6.75166959215993
CSGP 6.868799859401752
JD 8.923399310526852
SPLV 4.175863062590233
MDLZ 6.153884542017189
PEG 1.3176099357270563
MET 1.429205182040108
FTNT 9.265521908313895
GILD 10.672534506274095
STT -11.534078720169795
KO 9.519095546900546
GE -17.607870344125836
IVW -0.9875080234636859
SAIL 24.717354887259273
COUP -22.935454225339836
TD -6.52094268717361
SCHW -7.662003212292191
ACC 16.324271148291857
BAX -7.296720431707245
APH -1.0498138888742403
STLD -3.58399954241331
DOCU -16.320145153085164


In [ ]:
sorted_x = sorted(ticker_dict.items(), key=lambda x: x[1])

In [ ]:
for x in sorted_x:
    if abs(x[1]) > 15: 
        print(x[0], (round(x[1], 2)))